<a href="https://colab.research.google.com/github/Orson93/surprise_project/blob/master/CythonBlackScholesOptionPricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext cython

In [ ]:
# https://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb
# https://colab.research.google.com/drive/1oXcSCor9mH72N8NVeYX4Sl2jD0UWII2n#scrollTo=WaQrJS1gMnam
# https://github.com/ellisonbg/cythonoptions
# https://srdas.github.io/Papers/cython.pdf
# https://colab.research.google.com/drive/1oXcSCor9mH72N8NVeYX4Sl2jD0UWII2n#scrollTo=ooRjCqo0JpW5
# https://wstein.org/wiki/attachments/11(2f)480a/cython_cise_article.pdf

In [2]:
%%cython
cimport cython
from libc.math cimport exp, sqrt, pow, log, erf

@cython.cdivision(True)
cdef double std_norm_cdf(double x) nogil:
    return 0.5*(1+erf(x/sqrt(2.0)))

@cython.cdivision(True)
def black_scholes_cy(double s, double k, double t, double v,
                 double rf, double div, double cp):
    """Price an option using the Black-Scholes model.
    
    s : initial stock price
    k : strike price
    t : expiration time
    v : volatility
    rf : risk-free rate
    div : dividend
    cp : +1/-1 for call/put
    """
    cdef double d1, d2, optprice
    with nogil:
        d1 = (log(s/k)+(rf-div+0.5*pow(v,2))*t)/(v*sqrt(t))
        d2 = d1 - v*sqrt(t)
        optprice = cp*s*exp(-div*t)*std_norm_cdf(cp*d1) - \
            cp*k*exp(-rf*t)*std_norm_cdf(cp*d2)
    return optprice

In [3]:
black_scholes_cy(100.0, 100.0, 1.0, 0.2, 0.05, 0.0, 1)

10.450583572185565

In [4]:
from scipy import stats
import math

def black_scholes(s, k, t, v, rf, div, cp):
    """Price an option using the Black-Scholes model.
    
    s : initial stock price
    k : strike price
    t : expiration time
    v : volatility
    rf : risk-free rate
    div : dividend
    cp : +1/-1 for call/put
    """
    d1 = (math.log(s/k)+(rf-div+0.5*math.pow(v,2))*t)/(v*math.sqrt(t))
    d2 = d1 - v*math.sqrt(t)
    optprice = cp*s*math.exp(-div*t)*stats.norm.cdf(cp*d1) - \
        cp*k*math.exp(-rf*t)*stats.norm.cdf(cp*d2) 
    return optprice

In [5]:
black_scholes(100.0, 100.0, 1.0, 0.2, 0.05, 0.0, 1)

10.450583572185565

In [6]:
%timeit -n 250 black_scholes_cy(100.0, 100.0, 1.0, 0.3, 0.03, 0.0, -1)

The slowest run took 6.66 times longer than the fastest. This could mean that an intermediate result is being cached.
250 loops, best of 3: 386 ns per loop


In [7]:
%timeit -n 250 black_scholes(100.0, 100.0, 1.0, 0.3, 0.03, 0.0, -1)

250 loops, best of 3: 161 µs per loop
